## Load the Cleaned data

In [3]:
import pandas as pd
data_path = "/Users/agalyaayyadurai/PycharmProjects/Customer-Value-Insights/data/processed/cleaned_data.csv"
data = pd.read_csv(data_path)
print(data.head())

     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumWebVisitsMonth  AcceptedCmp3  \
0  04-09-2012       58       635  ...                  7             0   
1  08-03-2014       38        11  ...                  5             0   
2  21-08-2013       26       426  ...                  4             0   
3  10-02-2014       26        11  ...                  6             0   
4  19-01-2014       94       173  ...                  5             0   

   AcceptedCmp4  AcceptedCmp5  AcceptedCmp1  AcceptedCmp2  Complain  \
0             0

## Create new Features

In [4]:
# Calculate age
data['Age'] = 2024 - data['Year_Birth']

# Total children
data['Total_Children'] = data['Kidhome'] + data['Teenhome']

# Total spend
spend_features = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
data['Total_Spend'] = data[spend_features].sum(axis=1)

# Recency group
data['Recency_Group'] = pd.cut(data['Recency'], bins=4, labels=['Very Recent', 'Recent', 'Less Recent', 'Not Recent'])

print(data.head())

     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  AcceptedCmp1  AcceptedCmp2  Complain  \
0  04-09-2012       58       635  ...             0             0         0   
1  08-03-2014       38        11  ...             0             0         0   
2  21-08-2013       26       426  ...             0             0         0   
3  10-02-2014       26        11  ...             0             0         0   
4  19-01-2014       94       173  ...             0             0         0   

   Z_CostContact  Z_Revenue  Response  Age  Total_Childr

## Save the engineered data

In [5]:
engineered_data_path = "/Users/agalyaayyadurai/PycharmProjects/Customer-Value-Insights/data/processed/engineered_data.csv"
data.to_csv(engineered_data_path, index=False)